<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

<h1>Launch Sites Locations Analysis with Folium</h1>

Estimated time needed: **40** minutes

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.

<h2 id = Objectives>Objectives</h2>

This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.

<h2 id = libraries>Import of libraries</h2>

In [2]:
import folium
import pandas as pd

# We import essential functions of folium to do the markers 

# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

<h2 id = Tasks>Tasks of the use of Folium</h2>

<h3 id = task1>Task 1: Mark all launch sites on a map</h3>

First, let's try to add each site's location on a map using site's latitude and longitude coordinates

The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.

In [3]:
# We read from url the csv with all the location data
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

spacex_df=pd.read_csv(url)

We need to divide and group the information necessary from the dataframe

In [4]:
# Select relevant attributes
loc_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
# We group by the location site
launch_sites_df = loc_df.groupby(['Launch Site'], as_index = False).first()
# Reselect the attributes for the map
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
nasa_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [6]:
# Create a red circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a red circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
nasa_map.add_child(circle)
nasa_map.add_child(marker)

Now, let's add a circle for each launch site in data frame `launch_sites`

*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map

An example of folium.Circle:

`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`

An example of folium.Marker:

`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [7]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

for lat, lng, label in zip(launch_sites_df.Lat, launch_sites_df.Long, launch_sites_df['Launch Site']):
    folium.features.CircleMarker(
        [lat, lng],
        radius= 20, # define how big you want the circle markers to be
        color='blue',
        fill=True,
        popup=label,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(site_map)
    folium.map.Marker(
        [lat, lng],
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(30,30),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#0020d3;"><b>%s</b></div>' % label,
            )
    ).add_to(site_map)

site_map

Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?

**Answer:** No, all the launch sites are north to the equator line. This can be because they want to avoid to launch in another country. But according to the proximity, the launches are in the coast the closest to the equator line in the USA.

*   Are all launch sites in very close proximity to the coast?

**Answer:** All four sites has a close proximity to the coast, western and easter coast of the country of USA.

Also please try to explain your findings.

<h3 id = task2>Task 2: Mark the success/failed launches for each site on the map</h3>

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [8]:
spacex_df.groupby('Launch Site')['class'].mean()

Launch Site
CCAFS LC-40     0.269231
CCAFS SLC-40    0.428571
KSC LC-39A      0.769231
VAFB SLC-4E     0.400000
Name: class, dtype: float64

In [9]:
spacex_df.tail(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
46,43,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200.00,GTO,SES EchoStar,Success (drone ship),1,28.573255,-80.646895
47,44,2017-10-30,19:34:00,F9 B4 B1042.1,KSC LC-39A,Koreasat 5A,3500.00,GTO,KT Corporation,Success (drone ship),1,28.573255,-80.646895
48,54,2018-05-11,20:14:00,F9 B5 B1046.1,KSC LC-39A,Bangabandhu-1,3600.00,GTO,Thales-Alenia/BTRC,Success (drone ship),1,28.573255,-80.646895
49,45,2017-12-15,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205.00,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820
50,47,2018-01-08,1:00:00,F9 B4 B1043.1,CCAFS SLC-40,Zuma,3696.65,LEO,Northrop Grumman,Success (ground pad),1,28.563197,-80.576820
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.00,GTO,SES,Controlled (ocean),0,28.563197,-80.576820
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.00,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.576820
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.00,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.576820
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.00,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.576820
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.00,GTO,SES,No attempt,0,28.563197,-80.576820


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

Let's first create a `MarkerCluster` object

In [10]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [11]:
spacex_df['marker_color'] = pd.Series(['green' if value == 1 else 'red' for value in spacex_df['class']])

In [12]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows():
    a = 'Success ' + record['Date'] if record['class'] == 1 else 'Failure ' + record['Date'] 
    folium.Marker(
        location=[record['Lat'], record['Long']],
        icon = folium.Icon(color=record['marker_color']),
        color= record['marker_color'],
        popup = a
    ).add_to(marker_cluster)

site_map

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


<h3 id = task3>Task 3: Calculate the distances between a launch site to its proximities</h3>

Next, we need to explore and analyze the proximities of launch sites.

Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [13]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

First, we should deduce a formula to get the distance between two points in the globe, with the two pair of angles, latitude and longitude ($\lambda$,$\varphi$). If we have two points que can get two vectors then get the angle between them and then have the arc length or the distance.

$$
\vec{r}_1 = r(\cos{(\lambda_1)}\cos{(\varphi_1)}\hat{x} + \cos{(\lambda_1)}\sin{(\varphi_1)} \hat{y} + \sin{(\lambda_1)}\hat{z})
\\
\vec{r}_2 = r(\cos{(\lambda_2)}\cos{(\varphi_2)}\hat{x} + \cos{(\lambda_2)}\sin{(\varphi_2)} \hat{y} + \sin{(\lambda_2)}\hat{z})
$$

We do the dot product, where $\theta$ is the angle between the two vectors.
$$
\vec{r}_1 \cdot \vec{r}_2 = |\vec{r}_1||\vec{r}_2| \cos{(\theta)},\\
|\vec{r}_1||\vec{r}_2|(\cos{(\lambda_1)}\cos{(\lambda_2)}\cos{(\varphi_1)}\cos{(\varphi_2)} + \cos{(\lambda_1)}\cos{(\lambda_2)} \sin{(\varphi_1)}\sin{(\varphi_2)} + \sin{(\lambda_1)}\sin{(\lambda_2)})  = |\vec{r}_1||\vec{r}_2|\cos{(\theta)},\\
\cos{(\theta)} = \cos{(\lambda_1)}\cos{(\lambda_2)}(\cos{(\varphi_1)}\cos{(\varphi_2)} + \sin{(\varphi_1)}\sin{(\varphi_2)}) + \sin{(\lambda_1)}\sin{(\lambda_2)},\\
\cos{(\theta)} = \cos{(\lambda_1)}\cos{(\lambda_2)}\cos{(\varphi_2 - \varphi_1)} + \sin{(\lambda_1)}\sin{(\lambda_2)}.
$$

We could simplify the expression through the addition and the subtraction of a same factor "$\cos{(\lambda_1)}\cos{(\lambda_2)}$"

$$
\cos{(\theta)} = \cos{(\lambda_1)}\cos{(\lambda_2)}\cos{(\varphi_2 - \varphi_1)} +(- \cos{(\lambda_1)}\cos{(\lambda_2)} +  \cos{(\lambda_1)}\cos{(\lambda_2)})+\sin{(\lambda_1)}\sin{(\lambda_2)},\\
\cos{(\theta)} = \cos{(\lambda_1)}\cos{(\lambda_2)}(-1 + \cos{(\varphi_2 - \varphi_1)}) + \cos{(\lambda_2 - \lambda_1)}
$$

Then, we add in both sides to simplify the expression even more.

$$
\frac{1}{2}(1-\cos{(\theta)}) = \frac{1}{2}(1-(\cos{(\lambda_1)}\cos{(\lambda_2)}(-1 + \cos{(\varphi_2 - \varphi_1)}) + \cos{(\lambda_2 - \lambda_1)})),\\
\sin^2{(\frac{\theta}{2})} =  \cos{(\lambda_1)}\cos{(\lambda_2)} (\frac{1}{2})(1-\cos{(\varphi_2 - \varphi_1)}) + \frac{1}{2}- \frac{1}{2}\cos{(\lambda_2 - \lambda_1)}),\\
\sin^2{(\frac{\theta}{2})} = \cos{(\lambda_1)}\cos{(\lambda_2)}\sin^2{(\frac{\varphi_2 - \varphi_1}{2})} + \sin^2{(\frac{\lambda_2 - \lambda_1}{2})},
$$

where $\sin{(\frac{\theta}{2})}$ is the haversine formula, and we can get the angle through trigonometry.

$$
\sin{(\frac{\theta}{2})} = \sqrt{\cos{(\lambda_1)}\cos{(\lambda_2)}\sin^2{(\frac{\Delta \varphi}{2})} + \sin^2{(\frac{\Delta\lambda}{2})}} 
$$
but, by convenience is better to get the angle through the arctan function because it goes from $[-\pi, \pi]$, we get the cosine like the following

$$
\cos^2{(\frac{\theta}{2})} = 1 - \sin^2{(\frac{\theta}{2})} = 1 -\cos{(\lambda_1)}\cos{(\lambda_2)}\sin^2{(\frac{\Delta \varphi}{2})} + \sin^2{(\frac{\Delta\lambda}{2})}\\

\tan^2{(\frac{\theta}{2})} = \frac{\alpha}{1-\alpha},
$$

where $\alpha = \cos{(\lambda_1)}\cos{(\lambda_2)}\sin^2{(\frac{\Delta \varphi}{2})} + \sin^2{(\frac{\Delta\lambda}{2})}$

Then the angle of the distance between the two points is:

$$
\theta = 2\arctan{(\sqrt{\frac{\alpha}{1-\alpha}})},
$$
and the distance is the arc length
$$
d = R\theta = 2R\arctan{(\sqrt{\frac{\alpha}{1-\alpha}})},
$$

where $R$ is the radius of the earth in km

In [14]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1:float, lon1:float, lat2:float, lon2:float) -> float:
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [15]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
launch_site_lat,launch_site_lon = 28.56328, -80.5768
coastline_lat,coastline_lon = 28.56301, -80.56784

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [16]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
distance_marker = folium.Marker(
   [28.56301, -80.56784],
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 14; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
       )
   ).add_to(site_map)

site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [17]:
# lines=folium.PolyLine(locations=coordinates, weight=1)
coordinates =  [[28.56328, -80.5768],
                [28.56301, -80.56784]]
lines = folium.PolyLine(locations=coordinates, weight=4,color="#FF0000",
                        tooltip="Distance to the coastland")
site_map.add_child(lines)

site_map

In [19]:
# Initialize the coordinates for the places required
railroad = [28.562435,-80.58715]
road = [28.56517,-80.57106]
city = [28.54764,-81.38457]

# Create a list of list to do the markers and lines of distance
sites = [railroad,road,city]
names_sites = ['railroad','road','closest city']

# Do a loop to get all the sites in the map
for i,site in enumerate(sites):
    distance = calculate_distance(launch_site_lat, launch_site_lon, site[0], site[1])
    folium.Marker(
        site,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 24; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
            )
    ).add_to(site_map)
    folium.PolyLine(locations = [[28.56328, -80.5768],site], weight=4,color="#FF0000",
                    tooltip="Distance to " + names_sites[i]).add_to(site_map)


site_map